# Install Transformers Library

In [ ]:
! pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

# specify GPU
device = torch.device("cuda")

# Load Dataset

In [ ]:
%%time

df = pd.read_csv("data.csv")
df.head()

CPU times: user 910 ms, sys: 142 ms, total: 1.05 s
Wall time: 1.05 s


,label,text
0,1,WASHINGTON (Reuters) - The head of a conservat...
1,1,WASHINGTON (Reuters) - Transgender people will...
2,1,WASHINGTON (Reuters) - The special counsel inv...
3,1,WASHINGTON (Reuters) - Trump campaign adviser ...
4,1,SEATTLE/WASHINGTON (Reuters) - President Donal...


In [ ]:
df_small = df.loc[np.random.randint(0, df.shape[0], 500)]

In [ ]:
df_small['label'].mean()

0.51

## Acquire OpenAI API Endpoint

In [ ]:
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

### Create `word_embedding` Function

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

def word_embedding(text: str):
    output = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text,
        encoding_format="float"
    )

    return output.data[0].embedding

In [ ]:
df_small = df.loc[np.random.randint(0, df.shape[0], 500)]

In [ ]:
%%time

df_small['embedding'] = df_small.apply(lambda row: word_embedding(row['text']), axis=1)

CPU times: user 8.45 s, sys: 254 ms, total: 8.7 s
Wall time: 4min 24s


In [ ]:
df_small.head()

,label,text,embedding
13810,1,JAKARTA (Reuters) - The closure of I Gusti Ngu...,"[-0.014054704, -0.030279916, 0.006176567, 0.00..."
26837,0,Donald Trump s wife was the headline speaker o...,"[-0.03336349, -0.017549355, -0.018850768, -0.0..."
16302,1,MADRID (Reuters) - The regional parliament of ...,"[0.0071368287, 0.016498122, 0.014763569, -0.03..."
39047,0,"Houston, Texas trial lawyer and Democrat mega-...","[-0.020817684, 0.0014526903, 0.010799594, -0.0..."
20143,1,BERLIN (Reuters) - German Finance Minister Wol...,"[0.012651787, -0.021570904, 0.02550003, -0.013..."


In [ ]:
X = df_small['embedding'].values
y = df_small['label'].values

In [ ]:
X = np.array([np.array(X[i]) for i in range(len(X))])

In [ ]:
X.shape, y.shape

((500, 1536), (500,))

## Build `tensorflow` Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization, ReLU, Input
from tensorflow.keras.models import Model

# Input layer
inputs = Input(shape=(1536,))

# First dense layer
x = Dense(512)(inputs)
x = BatchNormalization()(x)
x = ReLU()(x)

# Second dense layer
x = Dense(256)(x)
x = BatchNormalization()(x)
x = ReLU()(x)

# Third dense layer
x = Dense(128)(x)
x = BatchNormalization()(x)
x = ReLU()(x)

# Output layer
outputs = Dense(1, activation='sigmoid')(x)

# Define the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1536)]            0         
                                                                 
 dense_12 (Dense)            (None, 512)               786944    
                                                                 
 batch_normalization_9 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 re_lu_9 (ReLU)              (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_10 (Ba  (None, 256)               1024      
 tchNormalization)                                           

In [ ]:
model.fit(X, y, epochs=10, validation_split=0.2, batch_size=32)

Epoch 1/10
13/13 [==============================] - 0s 12ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.0963 - val_accuracy: 0.9500
Epoch 2/10
13/13 [==============================] - 0s 8ms/step - loss: 3.3346e-04 - accuracy: 1.0000 - val_loss: 0.0974 - val_accuracy: 0.9600
Epoch 3/10
13/13 [==============================] - 0s 8ms/step - loss: 2.8714e-04 - accuracy: 1.0000 - val_loss: 0.0915 - val_accuracy: 0.9600
Epoch 4/10
13/13 [==============================] - 0s 9ms/step - loss: 3.5183e-04 - accuracy: 1.0000 - val_loss: 0.0860 - val_accuracy: 0.9600
Epoch 5/10
13/13 [==============================] - 0s 10ms/step - loss: 3.0963e-04 - accuracy: 1.0000 - val_loss: 0.0795 - val_accuracy: 0.9600
Epoch 6/10
13/13 [==============================] - 0s 10ms/step - loss: 3.3611e-04 - accuracy: 1.0000 - val_loss: 0.0728 - val_accuracy: 0.9700
Epoch 7/10
13/13 [==============================] - 0s 11ms/step - loss: 7.0096e-04 - accuracy: 1.0000 - val_loss: 0.0673 - val_accuracy:

## Performance Evaluation

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Assuming you have already trained your model and have X and y

# Make predictions
predictions = model.predict(X)

# Since the output layer uses sigmoid activation, we need to convert predictions to binary values
predictions_binary = (predictions > 0.5).astype(int)

# Flatten the predictions to match the shape of y
predictions_binary = predictions_binary.flatten()

# Generate the classification report
report = classification_report(y, predictions_binary)

# Display the classification report
print(report)

16/16 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       248
           1       0.99      1.00      1.00       252

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500



## Push to HF Cloud

In [ ]:
# Define name
nom = "news_embedding_clf"
path = f"folder_{nom}"

In [ ]:
%%capture
! pip install git+https://github.com/huggingface/huggingface_hub.git@main
! sudo apt -qq install git-lfs
! git config --global credential.helper store

In [ ]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write)

In [ ]:
from huggingface_hub import push_to_hub_keras

In [ ]:
push_to_hub_keras(model, "news-clf-model")

fingerprint.pb:   0%|          | 0.00/56.0 [00:00<?, ?B/s]

saved_model.pb:   0%|          | 0.00/203k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

variables.data-00000-of-00001:   0%|          | 0.00/3.83M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/eagle0504/news-clf-model/commit/d270456bd3023640b3273204b1c8e662363bfcf1', commit_message='Push Keras model using huggingface_hub.', commit_description='', oid='d270456bd3023640b3273204b1c8e662363bfcf1', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from huggingface_hub import from_pretrained_keras

reloaded_model = from_pretrained_keras('eagle0504/news-clf-model')

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

## Inference: Acquire Model from HF Cloud

In [ ]:
from huggingface_hub import from_pretrained_keras
import numpy as np
import tensorflow as tf

# Load the model
reloaded_model = from_pretrained_keras('eagle0504/news-clf-model')

# Prepare input data
# Ensure the input data is of dtype float32
input_data = np.random.rand(500, 1536).astype(np.float32)  # Replace with actual data

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
input_data = X.astype(np.float32)  # Replace with actual data

In [ ]:
# Make predictions
predictions = reloaded_model(input_data, training=False)

# Convert predictions to a NumPy array if necessary
predictions = predictions.numpy()


In [ ]:
# Since the output layer uses sigmoid activation, we need to convert predictions to binary values
predictions_binary = (predictions > 0.5).astype(int)

# Flatten the predictions to match the shape of y
predictions_binary = predictions_binary.flatten()

# Generate the classification report
report = classification_report(y, predictions_binary)

# Display the classification report
print(report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       248
           1       0.99      1.00      1.00       252

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500

